In [1]:
import sys
sys.path.append( '../vqe_in_dft' )

import numpy as np

# Background

The overlap matrix is:

$$S_{\mu \nu} =  \int d\vec{r}_{1} \phi_{\mu}(1)^{*}\phi_{\nu}(1)$$

- $\phi_{\mu}$ are basis functions (defined in basis set)


The unknown molecular orbitals $\psi_{i}$ are expanded as a linear expansion of the $K$ known basis functions:

$$ \psi_{i} =  \sum_{\mu=1}^{K} C_{\mu i} \psi_{\mu}$$


$C$ is a $K \times K$ matrix of expansion coefficients $C_{\mu i}$. The columns of $C$ describe the molecular orbitals!


We can find the total number of electrons $N$ in the system by:

$$ N =  2 \sum_{a}^{N/2}\int d\vec{r}  \bigg( \psi_{a}(\vec{r})^{*} \psi_{i}(\vec{r}) \bigg) =  2 \sum_{a}^{N/2} 1$$

- integral gives probablity of finding electron $a$ over all space (must be 1)
- summing over all electrons will give the total number of electrons

The charge density has the following definition:

$$\rho(\vec{r}) = 2 \sum_{a}^{N/2} \bigg( \psi_{a}(\vec{r})^{*} \psi_{i}(\vec{r}) \bigg)$$

- re-write using definition of $\psi_{i}=  \sum_{\mu=1}^{K} C_{\mu i} \phi_{\mu}$

$$\rho(\vec{r}) = 2 \sum_{a}^{N/2} \Bigg( \bigg[ \sum_{\nu}^{K} C_{\nu a}^{*} \phi_{\nu}(\vec{r})^{*} \bigg] \bigg[ \sum_{\mu}^{K} C_{\mu a}\phi_{\mu}(\vec{r}) \bigg] \Bigg)$$

- move things around

$$\rho(\vec{r}) = \sum_{\nu}^{K} \sum_{\mu}^{K} \Big( 2 \sum_{a}^{N/2} C_{\mu a} C_{\nu a}^{*} \Big) \phi_{\mu}(\vec{r}) \phi_{\nu}(\vec{r})^{*} $$

- which is 

$$\rho(\vec{r}) = \sum_{\mu, \nu}^{K} P_{\mu \nu} \phi_{\mu}(\vec{r}) \phi_{\nu}(\vec{r})^{*} $$


- $P_{\mu \nu}$ is known as the density matrix and is:

$$P_{\mu \nu} = 2 \sum_{a}^{N/2} C_{\mu a} C_{\nu a}^{*}$$

Therefore we can also find the total number of electrons in the system by:

$$ N =  2 \sum_{a}^{N/2}\int d\vec{r}  \bigg( \psi_{a}(\vec{r})^{*} \psi_{i}(\vec{r}) \bigg) =  \sum_{\nu}^{K} \sum_{\mu}^{K} \Big( 2 \sum_{a}^{N/2} C_{\mu a} C_{\nu a}^{*} \Big) \int d\vec{r} \phi_{\mu}(\vec{r})  \phi_{\nu}(\vec{r})^{*}$$

- This is simply:

$$N =  \sum_{\nu}^{K} \sum_{\mu}^{K} P_{\mu \nu} S_{\nu \mu}= \sum_{\mu}^{K} PS_{\mu \mu} = \mathcal{Tr}(PS)$$

- One can interpret $ PS_{\mu \mu}$ in the above equation as the number of electrons associated with $ \phi_{\mu}$
- This is a **Mulliken population analysis**

# Orbital Localization

- When we perform a SCF calculation, one gets an optimized C matrix
    - $C$ is a $K \times K$ matrix of expansion coefficients $C_{\mu i}$
    - The columns of $C$ describe the molecular orbitals!
    - MO i: $ \psi_{i} =  \sum_{\mu=1}^{K} C_{\mu i} \psi_{\mu}$
    
    
- These molecular orbitals are usually **delocalized**
    - non-negligible amplitude over the whole system, rather than only around some atom(s) or bond(s)

- But we know in QM that a given basis choice is NOT unique


- We can therefore perform a unitary rotation on molecular orbitals

$$ \psi_{i} U_{rot} =  \Big( \sum_{\mu=1}^{K} C_{\mu i} \psi_{\mu} \Big) U_{rot} = \psi_{i}^{new}$$
    
    
The idea is to use a rotation such that the resulting orbitals $\psi_{i}^{new}$ are as spatially localized as possible. 


The Pipek-Mezey (PM) [localization](https://notendur.hi.is/hj/papers/paperPipekmezey8.pdf) **maximizes the population charges on the atoms**:

$$ f (U_{rot}) = \sum_{A}^{N_{atoms}} \Bigg( Z_{A} -  \sum_{\mu \text{ on atom } A} PS_{\mu \mu} \Bigg)$$

### Method 1
- Given optimized $C$ coefficient matrix
    - which has been rotated to localize orbitals
    - (used to build localized density matrix)


- **Look through basis functions $\phi_{\mu}$ of the ACTIVE atoms**

    
- check the mulliken charge // mulliken population of the orbital
    - if above a certain threshold associate it to active system
    - otherwise put in the environment
 


To choose the active and enviroment subsystems we do the following:

1. Given a localized molecular orbs (localized C matrix), we take the absolute mag squared of the coefficients of the active part for a given localized orb and divide by the absolute mag squared of all the coefficents of a that orb... THis will give a value of how much the active system contributes to that orb.

2. Mathematically, for orbital $j$ 
    - remember MO orbs given by columns of C matrix
    - In equation below C matrix is the LOCALIZED form!


$$ \text{threshold} =  \frac{\sum_{\mu\in \text{active AO}}^{K} |C_{\mu j}|^{2}}{\sum_{\mu =1}^{K} |C_{\mu j}|^{2}}$$

## METHOD 2 - SPADE

WRITE NOTES TODO

In [2]:
### inputs
from pyscf import gto, scf

water_xyz = water = """3

H\t0.7493682\t0.0\t0.4424329
O\t0.0\t0.0\t-0.1653507
H\t-0.7493682\t0.0\t0.4424329
    """


basis = 'STO-3G'
charge=0
spin=0
full_system_mol = gto.Mole(atom= water_xyz[3:],
                      basis=basis,
                       charge=charge,
                       spin=spin,
                      )
full_system_mol.build()

HF_scf = scf.RHF(full_system_mol)
HF_scf.verbose=1
HF_scf.conv_tol = 1e-6
HF_scf.kernel()
###

-74.96445640381184

In [3]:
type(HF_scf)


pyscf.scf.hf.RHF

In [4]:
from localisation import localize_molecular_orbs

N_active_atoms=2
localization_method = 'ibo' # spade, boys, pipekmezey, ibo

(C_active, 
 C_enviro,
 C_loc_occ_full, 
 dm_active, 
 dm_enviro, 
 active_MO_inds, 
 enviro_MO_inds,
 C_loc_occ_and_virt, 
 active_virtual_MO_inds, 
 enviro_virtual_MO_inds) = localize_molecular_orbs(HF_scf,
                                                 N_active_atoms,
                                                 localization_method,
                                                 occ_THRESHOLD= 0.95,
                                                 virt_THRESHOLD=0.95,
                                                 sanity_check=True, 
                                                 run_virtual_localization= True
                                                 )

 Iterative localization: IB/P4/2x2, 5 iter; Final gradient 9.09e-10


In [5]:
dm_loc = 2* C_loc_occ_full@ C_loc_occ_full.conj().T
dm_std = 2* HF_scf.mo_coeff[:,HF_scf.mo_occ>0]@ HF_scf.mo_coeff[:,HF_scf.mo_occ>0].conj().T

print('want these C_loc and C_opt_std to be different:', not np.allclose(C_loc_occ_full,
                                                                         HF_scf.mo_coeff[:,HF_scf.mo_occ>0]))
print('want dm_std and dm_loc to be the SAME:',np.allclose(dm_std, dm_loc))

want these C_loc and C_opt_std to be different: True
want dm_std and dm_loc to be the SAME: True


# IMPORTANT orbital localization point!

When we performed orbital localization a unitary rotation was used... this left the generated density matrix unchanged. However, the molecular orbitals (C_matrix) will be DIFFERENT!

We therefore need to also **rotate the Fock matrix** otherwise the orbital energies no longer make any sense (even though the final energy will be the same)!


In [6]:
# here we see the localized and standard density matrices are the same!
print('want dm_std and dm_loc to be the SAME:',np.allclose(dm_std, dm_loc), '\n')


print('standard orb energies : \n')
print(np.diag(HF_scf.mo_coeff.conj().T @ HF_scf.get_fock() @ HF_scf.mo_coeff))
print(HF_scf.mo_energy)

print('\nLOCALIZED orb energies NOT making sense: \n')
print(np.diag(C_loc_occ_and_virt.conj().T @ HF_scf.get_fock() @ C_loc_occ_and_virt))
print(HF_scf.mo_energy)

# NOTE HOW THERE IS A DIFFERENCE WHEN LOCALIZED... To fix this we need to rotate the Fock matrix!

want dm_std and dm_loc to be the SAME: True 

standard orb energies : 

[-20.2461413   -1.26758708  -0.60956617  -0.45830837  -0.39267027
   0.60255579   0.72594124]
[-20.24614418  -1.26758942  -0.60956836  -0.45830668  -0.39267258
   0.60255701   0.72593768]

LOCALIZED orb energies NOT making sense: 

[ -2.75592512  -0.87779214  -0.87779214  -8.09943026 -10.36333352
   0.60255579   0.72594124]
[-20.24614418  -1.26758942  -0.60956836  -0.45830668  -0.39267258
   0.60255701   0.72593768]


# Define Fock matrix in new basis

How to get the operator that changes the basis?

1. Put the standard (unlocalized) and localized C matrix into orthogonal basis
    - $C_{\text{std}}^{\text{ORTHO}} = S^{0.5}C_{\text{std}}$
    - $C_{\text{loc}}^{\text{ORTHO}} = S^{0.5}C_{\text{loc}}$

2. Define change of basis
    - $U = \sum_{i \in occupied} | \psi_{i}^{\text{ORTHO, STD}} \rangle \langle \psi_{i}^{\text{ortho, loc}} | + \sum_{i \in virtual} | \psi_{i}^{\text{ORTHO, STD}} \rangle \langle \psi_{i}^{\text{ortho, STD}}| $ 
    - note this projects occupied standard ortho orbs onto localized orbs
    - keeps virtual orbs the same
        - if we localize the virtual orbs then we would need to use the same procedure as the occupied orbs (easy to include)

Note that $U$ is currently defined using ORTHOGONAL BASIS... we want it in our usual MO basis

Remembering that

$$C_{\text{std}}^{\text{ORTHO}} = S^{0.5}C_{\text{std}}$$

Clearly to get back to standard basis we do:

$$C_{\text{std}} = S^{-0.5} C_{\text{std}}^{\text{ORTHO}}$$


NOW as 


$$U C_{\text{loc}}^{\text{ORTHO}} =C_{\text{std}}^{\text{ORTHO}}$$

we can re-write this using our definitions:
- $C_{\text{std}}^{\text{ORTHO}} = S^{0.5}C_{\text{std}}$
- $C_{\text{loc}}^{\text{ORTHO}} = S^{0.5}C_{\text{loc}}$

$$U (S^{0.5}C_{\text{std}}) = (S^{0.5}C_{\text{loc}})$$

Now we want to get rid of $S^{0.5}$ on RHS so we times on left by $S^{-0.5}$

$$ S^{-0.5} U S^{0.5}C_{\text{std}} = C_{\text{loc}}$$

Therefore the change of basis operator in our non-ortho basis is simply:

$$ S^{-0.5} U S^{0.5}C_{\text{std}} = V$$

Where: $ V C_{\text{std}} = C_{\text{loc}}$


This is what the Get_orb_change_basis_operator function does!

In [7]:
from localisation import orb_change_basis_operator

V_change_basis = orb_change_basis_operator(HF_scf,
                                           C_loc_occ_and_virt,
                                           sanity_check=True)

In [8]:
print('V @ C_loc = C_std:', np.allclose(V_change_basis @C_loc_occ_and_virt,
                                            HF_scf.mo_coeff))

V @ C_loc = C_std: True


In [9]:
print('\nLOCALIZED orb energies WITH Fock change of basis : \n')
print(np.diag(C_loc_occ_and_virt.conj().T @ (V_change_basis.conj().T @ HF_scf.get_fock() @ V_change_basis) @ C_loc_occ_and_virt))
print(HF_scf.mo_energy)



LOCALIZED orb energies WITH Fock change of basis : 

[-20.2461413   -1.26758708  -0.60956617  -0.45830837  -0.39267027
   0.60255579   0.72594124]
[-20.24614418  -1.26758942  -0.60956836  -0.45830668  -0.39267258
   0.60255701   0.72593768]


In [10]:
# check S = V_dag S V
np.allclose(HF_scf.get_ovlp(),
            V_change_basis.conj().T @ HF_scf.get_ovlp() @ V_change_basis)

True

Overall what we done is the followin:

$$FC_{\text{std}} = S C_{\text{std}} E$$

$$F V^{\dagger} C_{\text{loc}} = S V^{\dagger} C_{\text{loc}} E$$

Left multiply by $V$:

$$ VF V^{\dagger} C_{\text{loc}} = V S V^{\dagger} C_{\text{loc}} E$$

$$ F_{new} C_{\text{loc}} = S C_{\text{loc}} E$$

- $F_{new} = VF V^{\dagger}$
- $S = V S V^{\dagger} $

In [11]:
def Get_new_Hcore(H_core, Unitary_rot):
    H_core_rot = Unitary_rot.conj().T @ H_core @Unitary_rot 
    return H_core_rot


def Get_new_Veff_HFock(pyscf_obj, Unitary, dm=None, check_result=False, hermi=1):
    
    # note this Veff has different definition to RKS DFT!
    
    if dm is None:
        if pyscf_obj.mo_coeff is not None:
            density_mat = pyscf_obj.make_rdm1(pyscf_obj.mo_coeff, pyscf_obj.mo_occ)
        else:
            density_mat = pyscf_obj.init_guess_by_1e()
    else:
        density_mat = dm
    
    vj, vk = pyscf_obj.get_jk(dm=density_mat, hermi=hermi)
    Veff = vj - vk * .5
    
#     Veff = pyscf_obj.get_veff(dm=density_mat)
    Veff_new = Unitary.conj().T @ Veff @ Unitary

    return Veff_new

In [12]:
### inputs
from pyscf import gto, scf


HF_scf_LOCALIZED = scf.RHF(full_system_mol)


# ############## include change of basis!
Hcore_std = HF_scf_LOCALIZED.get_hcore()
HF_scf_LOCALIZED.get_hcore = lambda *args: Get_new_Hcore(Hcore_std, 
                                                     Unitary_rot=V_change_basis)


HF_scf_LOCALIZED.get_veff = lambda mol=None, dm=None, dm_last=0, vhf_last=0, hermi=1: Get_new_Veff_HFock(
                                                                                    HF_scf_LOCALIZED, 
                                                                                       V_change_basis,
                                                                                        dm=dm,
                                                                                        hermi=hermi)
####


HF_scf_LOCALIZED.mo_coeff = C_loc_occ_and_virt
HF_scf_LOCALIZED.mo_occ = HF_scf.mo_occ


###
HF_scf_LOCALIZED.energy_tot()

-74.96445640381206

In [13]:
print('HF energies match original cannonical calc: ', np.isclose(HF_scf_LOCALIZED.energy_tot(), 
                               HF_scf.e_tot))

HF energies match original cannonical calc:  True


In [14]:
F_new_basis = HF_scf_LOCALIZED.get_fock()

orb_energies_new_basis = np.diag(C_loc_occ_and_virt.conj().T @ F_new_basis @ C_loc_occ_and_virt)


print('do orbital energies match: ', np.allclose(HF_scf.mo_energy,
                                                orb_energies_new_basis))



do orbital energies match:  True
